# Data Aggregation
As a first step in the Places to Go Demo, we will need static venue data to create reccomendations from. In production, our venue sources will be managed by a Web Scraper bot that will handle crawling social media and updating the list based on activity. For now, we will populate a static set of 2,500 locations, which will be sourced from 5 cities. 

The five cities that have been requested by the client for the demo are:
- **New York**
- **Scottsdale**
- **Miami**
- **Los Angeles**
- **Chicago**

We will use the Yelp API to gather the top 500 rated locations in each city. We will then feed the `name` and `categories` field of each response to the AI model, which will seek to associate each venue with a list of keywords.

We will need to take the following steps to achieve our task:
1. Gather JSON objects for top 500 locations in each city
2. Extract exhaustive list of all categories from the 2,500 locations
3. Provide list of ChatGPT and prompt it to create a list of 20 keywords for each archetype
4. Design prompt for associating businesses with keywords based on `name` and `categories` field 
5. Run list of 2,500 businesses and store results in a JSON file.

In [9]:
import os
from dotenv import load_dotenv
load_dotenv("../.env")

YELP_API_KEY = os.getenv("YELP_API_KEY")
TRIP_ADVISOR_API_KEY = os.getenv("TRIP_ADVISOR_API_KEY")

## 1. Gather JSON Data of Locations
We want to start by using the `/businesses/search` endpoint of the Yelp Fusion API to gather the top 500 rated locations in each of our 5 cities. We will store these responses directly in JSON files to retrieve for future steps.

In [14]:
SEARCH_TERMS = ["tour", "activity", "experience", "resturant", "bar", "nightclub", "explore", "adventure", "museum", "nature"]

In [16]:
import requests
from tqdm import tqdm
from typing import Optional

# List of cities to search
CITIES = ["New%20York%20City", "Scottsdale", "Miami", "Los%20Angeles", "Chicago"]
CITY_CODES = ["NYC", "SCOTTSDALE", "MIAMI", "LA", "CHICAGO"]

CITY_TO_CODE = dict(zip(CITIES, CITY_CODES))

# Yelp Fusion API URL
API_URL = "https://api.yelp.com/v3"
BUSINESS_SEARCH_ENDPOINT = "/businesses/search"

# Search Params For API Request
LIMIT = 50
SORT_BY = "best_match"
LOCALE = "en_US"

# Authorization
HEADERS = {
    "Authorization": "Bearer " + YELP_API_KEY,
}

def request_city_data(city: str):
    """Request data from Yelp API for a given city"""
    base_url = f"{API_URL}{BUSINESS_SEARCH_ENDPOINT}?location={city}&limit={LIMIT}&sort_by={SORT_BY}&local={LOCALE}"
    data = []
    for i, search_term in enumerate(SEARCH_TERMS):
        url = base_url + f"&term={search_term}"
        results = requests.get(url, headers=HEADERS).json()
        # Add the city code to the data
        for result in results['businesses']: result['city'] = city
        data.extend(results["businesses"])

        # Reset the cursor to not interrupt the tqdm progress bar
        print(f"Found {len(results['businesses'])} results for {city} with term {search_term}")
    
    # Log City Results
    print(f"Found {len(data)} results for {city}")
    return data

def aggregate_city_data():
    """Aggregate data from all cities"""
    data = []
    for city in tqdm(CITIES):
        data.extend(request_city_data(city))

    print("\r", end="")
    return data

In [17]:
activity_data = aggregate_city_data()

  0%|          | 0/5 [00:00<?, ?it/s]

Found 50 results for New%20York%20City with term tour
Found 50 results for New%20York%20City with term activity
Found 50 results for New%20York%20City with term experience
Found 50 results for New%20York%20City with term resturant
Found 50 results for New%20York%20City with term bar
Found 50 results for New%20York%20City with term nightclub
Found 50 results for New%20York%20City with term explore
Found 50 results for New%20York%20City with term adventure
Found 50 results for New%20York%20City with term museum


 20%|██        | 1/5 [00:07<00:30,  7.53s/it]

Found 50 results for New%20York%20City with term nature
Found 500 results for New%20York%20City
Found 50 results for Scottsdale with term tour
Found 50 results for Scottsdale with term activity
Found 50 results for Scottsdale with term experience
Found 50 results for Scottsdale with term resturant
Found 50 results for Scottsdale with term bar
Found 50 results for Scottsdale with term nightclub
Found 50 results for Scottsdale with term explore
Found 50 results for Scottsdale with term adventure
Found 50 results for Scottsdale with term museum


 40%|████      | 2/5 [00:14<00:21,  7.08s/it]

...

Found 50 results for Miami with term nature
Found 500 results for Miami
Found 50 results for Los%20Angeles with term tour
Found 50 results for Los%20Angeles with term activity
Found 50 results for Los%20Angeles with term experience
Found 50 results for Los%20Angeles with term resturant
Found 50 results for Los%20Angeles with term bar
Found 50 results for Los%20Angeles with term nightclub
Found 50 results for Los%20Angeles with term explore
Found 50 results for Los%20Angeles with term adventure
Found 50 results for Los%20Angeles with term museum


 80%|████████  | 4/5 [00:30<00:07,  7.84s/it]

Found 50 results for Los%20Angeles with term nature
Found 500 results for Los%20Angeles
Found 50 results for Chicago with term tour
Found 50 results for Chicago with term activity
Found 50 results for Chicago with term experience
Found 50 results for Chicago with term resturant
Found 50 results for Chicago with term bar
Found 50 results for Chicago with term nightclub
Found 50 results for Chicago with term explore
Found 50 results for Chicago with term adventure
Found 50 results for Chicago with term museum


100%|██████████| 5/5 [00:37<00:00,  7.59s/it]

Found 50 results for Chicago with term nature
Found 500 results for Chicago


In [22]:
import json
# APPEND New Locations to Location Data -- DANGEROUS

with open("../data/searched_location_data.json", "r", encoding="utf-8") as f:
    location_data = json.load(f)

total_locations = activity_data + location_data
total_location_ids = list(set([location['id'] for location in total_locations]))

locations = []
for _id in total_location_ids:
    for location in total_locations:
        if location['id'] == _id:
            locations.append(location)
            break

with open("../data/searched_location_data.json", "w", encoding="utf-8") as f:
    for loc in locations:
        loc['city_code'] = CITY_TO_CODE[loc['city']]
    json.dump(locations, f, ensure_ascii=False, indent=4)

## 3. Web Scraping Script
Run the webscraping script to get the reviews for each business.

## 4. Prompt Engineering
Now that we have our keywords set, we need to do some prompt engineering to create a GPT-3.5-Turbo prompt which associates a venue with a set of our keywords. To do this, there are a few considerations we must make:
- Prompt must provide the list of keywords to the model
- Model must accurately associate keywords with venues according to product needs
- Want to process as many venues in one prompt as possible

To provide the LLM with the list of keywords, we will simply provide them in the system prompt. For token efficiency, we may try to cram as many venues as possible into every prompt, so we limit the number of times we have to send a system prompt.

In order to get accurate results without fine-tuning, we should take a few-shot approach, to do this, we will use ChatGPT to do ~20 locations, and we will then use these as an example for each prompt we send.

Finally, we should try to jam as many tokens as possible into each prompt. We have 16k tokens to work with as a context window. We can use the examples to determine the optimal number of locations to use per prompt.

In [69]:
import json

with open("../scrape/locations_finished.json", "r", encoding="utf-8") as f:
    location_data = json.load(f)
    trimmed_location_data = []
    for location in location_data:
        trimmed_location_data.append({
            "id": location['id'],
            "name": location['name'],
            "city": location['city_code'],
            'rating': location['rating'],
            "reviews": location['reviews'],
        })
    location_data = trimmed_location_data
    del trimmed_location_data


2813

In [70]:

# Add an embed term to each location
for loc in location_data:
    reviews = '\n'.join([f"{list(review.keys())[0]}:\n{list(review.values())[0]}" for review in loc['reviews']])
    term = f"{loc['name']}\n\n{reviews}"
    loc['embed_term'] = term

embed_terms = [location['embed_term'] for location in location_data]

In [71]:
len(embed_terms)

2813

In [73]:
import pandas as pd
import numpy as np
from openai import OpenAI

from typing import List

categories = ["Restaurant", "Activity", "Museum", "Outdoor Exploration", "Shopping", "Nightlife", "Historical Site", "Amusement Park", "Experience", "Relaxation"]


class CategoryVectorstore():

    def __init__(self):
        embeddings = self._embed(categories)
        data = [{"category": category, "embedding": embedding} for category, embedding in zip(categories, embeddings)]
        self.vectorstore = pd.DataFrame(data)

    def _embed(self, terms: List[str]) -> List[List[float]]:
        response = OpenAI().embeddings.create(
            model="text-embedding-ada-002",
            input=terms,
        )
        return [result.embedding for result in response.data]

    def _search(self, embedding: List[float]) -> str:
        """Search for the closest category to a given embedding"""
        vectorstore = self.vectorstore.copy()
        vectorstore['score'] = vectorstore.embedding.apply(lambda x: np.dot(x, embedding))
        vectorstore.sort_values(by="score", ascending=False, inplace=True)

        category = vectorstore.iloc[0].category
        return category

    def get_categories(self, terms: List[str]) -> List[str]:
        """Get the categories for a list of terms"""
        embeddings = self._embed(terms)
        return [self._search(embedding) for embedding in embeddings]

category_vectorstore = CategoryVectorstore()
categories = [*category_vectorstore.get_categories(embed_terms[:1000]), *category_vectorstore.get_categories(embed_terms[1000:2000]), *category_vectorstore.get_categories(embed_terms[2000:])]

['Outdoor Exploration',
 'Restaurant',
 'Nightlife',
 'Nightlife',
 'Outdoor Exploration',
 'Museum',
 'Nightlife',
 'Nightlife',
 'Restaurant',
 'Museum',
 'Museum',
 'Restaurant',
 'Nightlife',
 'Nightlife',
 'Experience',
 'Nightlife',
 'Outdoor Exploration',
 'Museum',
 'Nightlife',
 'Outdoor Exploration',
 'Restaurant',
 'Nightlife',
 'Outdoor Exploration',
 'Outdoor Exploration',
 'Nightlife',
 'Museum',
 'Restaurant',
 'Nightlife',
 'Outdoor Exploration',
 'Experience',
 'Outdoor Exploration',
 'Restaurant',
 'Restaurant',
 'Nightlife',
 'Nightlife',
 'Restaurant',
 'Outdoor Exploration',
 'Outdoor Exploration',
 'Experience',
 'Museum',
 'Shopping',
 'Museum',
 'Amusement Park',
 'Nightlife',
 'Experience',
 'Nightlife',
 'Amusement Park',
 'Experience',
 'Restaurant',
 'Outdoor Exploration',
 'Outdoor Exploration',
 'Restaurant',
 'Restaurant',
 'Nightlife',
 'Relaxation',
 'Experience',
 'Restaurant',
 'Outdoor Exploration',
 'Museum',
 'Outdoor Exploration',
 'Nightlife',
 '

In [75]:
from keywords import KeywordVectorstore

vs = KeywordVectorstore()

relevant_keywords = [*vs.get_keywords(embed_terms[:1000]), *vs.get_keywords(embed_terms[1000:2000]), *vs.get_keywords(embed_terms[2000:])]


In [76]:
location_and_data = zip(relevant_keywords, categories, location_data)

cypher_entities = []
for i, data in enumerate(location_and_data):
    kewords, category, location = data
    cypher_entities.append({
        'venue': {
            'id': location['id'],
            'name': location['name'],
            'city': location['city'],
            'category': category,
            'rating': location['rating'],
        },
        'keywords': kewords,
    })
for entity in cypher_entities:
    if entity['venue']['category'] == "Not Vacation Related":
        print(entity)

In [78]:
with open("../data/cypher_entities.json", "w", encoding="utf-8") as f:
    json.dump(cypher_entities, f, ensure_ascii=False, indent=4)